In [1]:
import numpy as np
import pandas as pd
import scipy.stats
from scipy.stats import truncnorm
from pandas import DataFrame as df
import scipy.integrate as integrate
from scipy.optimize import minimize     # GMM optimization
from scipy.misc import derivative      # Jacobian Estimation
import warnings
warnings.filterwarnings(action='ignore')

# Part 0: Data Preparation

Before jumping into the main questions, in this part I will modify the given dataset to go through the given analysis. 
Structure of the given dataset requires imputation procedure for the following two reasons.  

<b>1) To aggregate the dataset with two different regimes </b>  
Throughout the problem set, we will assume that the random sample, log income level, follows the normal distribution. 

$$upto\,panel\, year\, 2009 $$
<img src="https://github.com/Ian-JY-Kim/Metrics/blob/main/pre_period.png?raw=true" width = "400" height ="300"></img>
Above graph shows the naive shape of the income distribution upto panel year 2009, where I used the bins rather than logged income level. I can check that the assumption of normal distribution is plausible. Since after transforming the income level with log function, the shape skewed to the right will come down.  

$$from\,panel\, year\, 2010 $$
<img src="https://github.com/Ian-JY-Kim/Metrics/blob/main/post_period.png?raw=true" width = "400" height ="300"></img> However, assuming the normal distribution with the post regime dataset seems not plausible. It is because the data shows us a highly skewed distribution resembles a pareto distribution rather than a normal distribution. This is due to the fact that after panel year 2010, high earners' income data is top coded at the bin 27, rather than 30

To aggregate the the data with two different regime, I will estimate the first and second moment of the normal distribution using the pre regime data (upto panel year 2009). And by assuming the normal distribution with the estimated parameters, I will calculate the probability of one's income goes into each bin between 27 and 30 conditional on one's income falls above bin 27. Using the calculated probability, I will impute the income bin 27, 28, 29, 30 of the post regime dataset. And then, I will aggregate the dataset


<b>2) To deal with the top coded data, especially with the top earners in bin 30 </b>  
Apart from the aggregation issue, to deal with the three different assumption which are  
(a) Assuming each household earns the lowest income in each bin  
(b) Assuming each household earns the highes income in each bin  
(c) Assuming each household earns the mean income in each bin  
in question 7, I will go through another imputation procedure. Because, without imputation, we can not work with assumption (b) and (c).

Since we already aggregated our dataset above, we don't have any other dataset to retrieve the first and second moment of the distribution from. Therefore, I will estimate the first and second moment using the data in bins from 1 to 29.  To estimate the moments, I will use the mean income level ($= \frac{highest+lowest}{2}$) for each bin for the assumption (b) and the highest income value for the assumption (c). 

And from there, for the assumption (b), I will calculate the mean value of bin 30.  
And for the assumption (c), I will draw 354410 many random sample from the truncated normal distribution with the range of bin 30 to find the maximum value.


## Aggregate the Data

In [2]:
dataset = pd.read_csv("simulated_income_data.csv")
dataset_pre = dataset[dataset['panel_year'] <= 2009]
dataset_post = dataset[dataset['panel_year'] > 2009]
dataset_pre = df(dataset_pre.groupby('household_income')['projection_factor'].sum())
dataset_pre = dataset_pre.reset_index()

income_amount = [0, 5000, 8000, 10000, 12000, 15000, 20000, 25000, 30000, 35000, 40000, 
                45000, 50000, 60000, 70000, 100000, 125000, 150000, 200000]
dataset_pre['income_amount'] = income_amount 
total_projection = np.array(dataset_pre['projection_factor']).sum()
dataset_pre['weight'] = dataset_pre['projection_factor']/total_projection
log_income_amount = []
for i in dataset_pre['income_amount']:
    if i == 0:
        log_income_amount.append(0)
    else:
        log_income_amount.append(np.log(i))
dataset_pre['log_income_amount'] = log_income_amount
mean_hat = np.inner(np.array(dataset_pre['weight']), np.array(dataset_pre['log_income_amount']))
sigma_sq_hat = np.inner(np.array(dataset_pre['weight']), (np.array(dataset_pre['log_income_amount'])-mean_hat)**2)

In [3]:
def norm_pdf(beta):
    return scipy.stats.norm(mean_hat,sigma_sq_hat**(1/2)).pdf(beta)
dataset_post = df(dataset_post.groupby('household_income')['projection_factor'].sum())
dataset_post = dataset_post.reset_index()

prob_above27 = integrate.quad(norm_pdf, np.log(100000), 100)[0]
prob_27 = integrate.quad(norm_pdf, np.log(100000), np.log(125000))[0] / prob_above27
prob_28 = integrate.quad(norm_pdf, np.log(125000), np.log(150000))[0] / prob_above27
prob_29 = integrate.quad(norm_pdf, np.log(150000), np.log(200000))[0] / prob_above27
prob_30 = integrate.quad(norm_pdf, np.log(200000), 100)[0] / prob_above27

imputed_27 = int(627298 * prob_27)
imputed_28 = int(627298 * prob_28)
imputed_29 = int(627298 * prob_29)
imputed_30 = int(627298 * prob_30)

household_income = list(dataset_pre['household_income'])
imputed_list = list(dataset_post['projection_factor'])[:-1] + [imputed_27, imputed_28, imputed_29, imputed_30]
imputed_dataset_post = df({'household_income': household_income, 'projection_factor_post': imputed_list})
master = dataset_pre.merge(imputed_dataset_post)
master['projection_total'] = master['projection_factor'] + master['projection_factor_post']
master['weight'] = master['projection_total']/np.array(master['projection_total']).sum()
aggregated_dataset = master[['household_income', 'projection_total','weight', 'income_amount', 'log_income_amount']]
aggregated_dataset

,household_income,projection_total,weight,income_amount,log_income_amount
0,3,46530,0.013407,0,0.000000
1,4,38061,0.010967,5000,8.517193
2,6,50001,0.014407,8000,8.987197
3,8,42577,0.012268,10000,9.210340
4,10,56830,0.016374,12000,9.392662
5,11,127104,0.036622,15000,9.615805
6,13,193586,0.055778,20000,9.903488
7,15,93956,0.027072,25000,10.126631
8,16,158063,0.045543,30000,10.308953
9,17,188202,0.054227,35000,10.463103


## Impute the Mean and Maximum Income of Bin 30

In [4]:
data_upto29 = aggregated_dataset[aggregated_dataset['household_income'] <= 29]
income_mean = [2500, 6500, 9000, 11000, 13000, 17500, 22500, 27500, 32500, 37500, 42500, 47500, 55000, 65000,
              85000, 112500, 137500, 175000]
income_max = [5000, 8000, 10000, 12000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 60000, 70000, 100000,
             125000, 150000, 200000]

In [5]:
data_upto29['income_amount_mean'] = income_mean
data_upto29['income_amount_highest'] = income_max
data_upto29['log_income_mean'] = [np.log(i) for i in data_upto29['income_amount_mean']]
data_upto29['log_income_highest'] = [np.log(i) for i in data_upto29['income_amount_highest']]
data_upto29['weight_upto29'] = data_upto29['projection_total']/np.array(data_upto29['projection_total']).sum()

mean_hat_ass_b = np.inner(np.array(data_upto29['weight_upto29']), np.array(data_upto29['log_income_mean']))
sigma_sq_hat_ass_b = np.inner(np.array(data_upto29['weight_upto29']), (np.array(data_upto29['log_income_mean'])-mean_hat_ass_b)**2)

mean_hat_ass_c = np.inner(np.array(data_upto29['weight_upto29']), np.array(data_upto29['log_income_highest']))
sigma_sq_hat_ass_c = np.inner(np.array(data_upto29['weight_upto29']), (np.array(data_upto29['log_income_highest'])-mean_hat_ass_b)**2)

norm_pdf_mean = lambda x: scipy.stats.norm(mean_hat_ass_b,sigma_sq_hat_ass_b**(1/2)).pdf(x)
condition = integrate.quad(norm_pdf_mean, np.log(200000), 100)[0]
f = lambda x: x * (scipy.stats.norm(mean_hat_ass_b,sigma_sq_hat_ass_b**(1/2)).pdf(x)) / condition 
mean_value_bin30 = integrate.quad(f, np.log(200000), 100)[0]

In [6]:
def get_truncated_normal(mean, sd, low, upp):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)
X = get_truncated_normal(mean_hat_ass_c, sigma_sq_hat_ass_c**1/2, np.log(200000), upp=100)

candidates = []
for i in range(100):
    random_draw = X.rvs(354410)
    candidates.append(random_draw.max())

max_value_bin30 = np.array(candidates).mean()

income_mean30 = [np.log(i) for i in income_mean] + [mean_value_bin30]
income_max30 = [np.log(i) for i in income_max] + [max_value_bin30]
aggregated_dataset['log_income_mean'] = income_mean30
aggregated_dataset['log_income_max'] = income_max30
aggregated_dataset

,household_income,projection_total,weight,income_amount,log_income_amount,log_income_mean,log_income_max
0,3,46530,0.013407,0,0.000000,7.824046,8.517193
1,4,38061,0.010967,5000,8.517193,8.779557,8.987197
2,6,50001,0.014407,8000,8.987197,9.104980,9.210340
3,8,42577,0.012268,10000,9.210340,9.305651,9.392662
4,10,56830,0.016374,12000,9.392662,9.472705,9.615805
5,11,127104,0.036622,15000,9.615805,9.769956,9.903488
6,13,193586,0.055778,20000,9.903488,10.021271,10.126631
7,15,93956,0.027072,25000,10.126631,10.221941,10.308953
8,16,158063,0.045543,30000,10.308953,10.388995,10.463103
9,17,188202,0.054227,35000,10.463103,10.532096,10.596635


----

# Part 1: MoM

## Q1. Log Normal

Let a log normal distribution is parametrized with $\mu$ and $\sigma$  

Then the first moment of log normal distribution becomes $ exp[\mu + \frac{\sigma^2}{2}]$ and the second moment becomes $(exp[\sigma^2]-1) exp[2\mu + \sigma^2]$  

Now, let's define the sample analogy of the first two moments with $\overline{Y_n} := \frac{1}{n}\sum_i^n y_i$ and $S^2 := \frac{1}{n-1}\sum_i^n(y_i - \overline{Y_n})^2$ 

Then, we have $S^2 = (exp[\hat{\sigma}^2]-1)\overline{Y_n}^2$, From which, we get $\hat{\sigma}^2 = ln(\frac{S^2}{\overline{Y_n}^2} + 1)$  

Then, from the $\overline{Y_n} = exp[\hat{\mu} + \frac{\hat{\sigma}^2}{2}]$ We get $\hat{\mu} = ln\overline{Y_n} - \frac{1}{2}ln(\frac{S^2}{\overline{Y_n}^2} + 1)$

In [7]:
sample_mean = np.inner(np.array(aggregated_dataset['weight']), np.array(aggregated_dataset['income_amount']))
temp1 = (np.array(aggregated_dataset['income_amount']) - sample_mean)**2
sample_variance = np.inner(np.array(aggregated_dataset['weight']), temp1)
sigma_sq_hat = np.log(sample_variance/(sample_mean**2) + 1)
mu_hat = np.log(sample_mean) - sigma_sq_hat/2

print("estimated mean: ", mu_hat)
print("estimated variance: ", sigma_sq_hat)

estimated mean:  10.998012204189621
estimated variance:  0.43773086897760277


## Q2. Normal Distribution

under normal distribution, we can estimate first and second moment with its sample analogy

In [161]:
MoM_mu = np.inner(np.array(aggregated_dataset['weight']), np.array(aggregated_dataset['log_income_amount']))
temp2 = (np.array(aggregated_dataset['log_income_amount']) - mu_hat2)**2
MoM_sigma = np.inner(np.array(aggregated_dataset['weight']), temp2)**(1/2)

print("estimated mean: ", MoM_mu)
print("estimated variance: ", MoM_sigma)

estimated mean:  10.79578469735779
estimated variance:  1.496574725684537


## Q3. Standard Error working with $lny_i$

<b>Mean Estimator</b>  

under a normal distribution assumption, sample analogies become the moment estimators for the first and second moment.  
Then, we can derive variance of mean estimator as follows,  

$$Var(\overline{Y_n}) = Var(\frac{1}{n}\sum_i^n\ln y_i)= \frac{1}{n^2}Var(\sum_i^n\ln y_i) = \frac{1}{n^2}nVar(\ln y_i) = \frac{\sigma^2}{n}$$

Hence by pluggin in the $S^2$ into $\sigma^2$, we can get an estimate for the standard error for the mean estimator, which is $\frac{S}{\sqrt{n}}$ 


<b>Variance Estimator</b>  

Note that $\frac{(n-1)S^2}{\sigma^2} \sim \chi_{n-1}^2$ 

Since the chi-squared distribution with n-1 degrees of freedom has a variance of $2(n-1)$, the variance of variance estimator becomes  

$$Var(S^2) = Var(\frac{\sigma^2 \chi_{n-1}^2}{n-1})= \frac{2\sigma^4}{n-1}$$

Hence by plugging in the $S^2$ into $\sigma^2$, we get an estimate for the standard error for the variance estimator, which is $S^2\sqrt{\frac{2}{n-1}}$


In [137]:
n = np.array(aggregated_dataset['projection_total']).sum()
MoM_std_err_mean = (sigmasq_hat2/n)**(1/2)
MoM_std_err_var = sigmasq_hat2 * (2/(n-1))**(1/2)

print("estimated standard error of MoM mean esimator: ", MoM_std_err_mean)
print("estimated standard error of MoM variance esimator: ", MoM_std_err_var)

estimated standard error of MoM mean esimator:  0.0008033272450482495
estimated standard error of MoM variance esimator:  0.0017002232994797742


# Part 2: MLE

## Q.4~Q.6

Due to the invariance property of MLE, we may concentrate on the logged income data and work with the normal distribution.  

I will the aggregated data I made above. And I will calculate the log-likelihood using the weight value 

## q4.  
$pr(\ln y_1, \cdots, \ln{y_2} | \mu, \sigma) = \prod_{i=1}^{n} \frac{1}{\sigma \sqrt{2\pi}} exp[-\frac{1}{2}(\frac{\ln y_i-\mu}{\sigma})^2]$


## q5.
$\mathcal{L}(\mu, \sigma ; \ln y_1, \cdots, \ln y_n) = -n \ln (\sigma \sqrt{2\pi}) - \frac{1}{2\sigma^2}\sum_{i=1}^{n}(\ln y_i - \mu)^2 = -\frac{1}{2}n\ln 2\pi - \frac{1}{2}n\ln{\sigma^2}- \frac{1}{2\sigma^2}\sum_{i=1}^{n}(\ln y_i - \mu)^2$

## q6.
score for $\mu$ (and $\sigma^2$) is a partial derivative of the log-likelihood w.r.t $\mu$ (and $\sigma^2$)  
hence score for $\mu = \frac{\partial \mathcal{L}}{\partial \mu} = \frac{1}{\sigma^2}\sum_{i=1}^{n}(\ln y_i - \mu)$  
and score for $\sigma^2 = \frac{\partial \mathcal{L}}{\partial \sigma^2} = -\frac{n}{2}  \frac{1}{\sigma^2} + \frac{1}{2} \frac{1}{(\sigma^2)^2}\sum_{i=1}^{n}(\ln y_i - \mu)^2$  
hence the score vector, which is an overall gradient becomes $s = \begin{bmatrix} \frac{1}{\sigma^2}\sum_{i=1}^{n}(\ln y_i - \mu)\\-\frac{n}{2} \frac{1}{\sigma^2} + \frac{1}{2} \frac{1}{(\sigma^2)^2}\sum_{i=1}^{n}(\ln y_i - \mu)^2 \end{bmatrix}$ 

## Q.7

Recall the first order derivative w.r.t each parameter:  

$$\frac{\partial \mathcal{L}}{\partial \mu} = \frac{1}{\sigma^2}\sum_{i=1}^{n}(\ln y_i - \mu)\\\  
\frac{\partial \mathcal{L}}{\partial \sigma^2} = -\frac{n}{2}  \frac{1}{\sigma^2} + \frac{1}{2} \frac{1}{(\sigma^2)^2}\sum_{i=1}^{n}(\ln y_i - \mu)^2$$  

Under $\widehat{\mu}_{MLE}, \widehat{\sigma^2}_{MLE}$ we require
$$\frac{1}{\widehat{\sigma^2}_{MLE}}\sum_{i=1}^{n}(\ln y_i - \widehat{\mu}_{MLE}) = 0 \\
-\frac{n}{2}  \frac{1}{\widehat{\sigma^2}_{MLE}} + \frac{1}{2} \frac{1}{(\widehat{\sigma^2}_{MLE})^2}\sum_{i=1}^{n}(\ln y_i - \widehat{\mu}_{MLE})^2 = 0$$

From which, we get the following estimates (provided that $\sigma^2 \neq 0$),  
$$ \widehat{\mu}_{MLE} = \frac{1}{n}\sum_{i=1}^{n}\ln y_i \\
 \widehat{\sigma^2}_{MLE} = \frac{1}{n}\sum_{i=1}^{n}(\ln y_i-\widehat{\mu}_{MLE})^2 $$


In [131]:
mu_lowest = np.inner(np.array(aggregated_dataset['weight']), np.array(aggregated_dataset['log_income_amount']))
mu_mean = np.inner(np.array(aggregated_dataset['weight']), np.array(aggregated_dataset['log_income_mean']))
mu_highest = np.inner(np.array(aggregated_dataset['weight']), np.array(aggregated_dataset['log_income_max']))

sigma_sq_lowest = np.inner(np.array(aggregated_dataset['weight']), (np.array(aggregated_dataset['log_income_amount'])-mu_lowest)**2)
sigma_sq_mean = np.inner(np.array(aggregated_dataset['weight']), (np.array(aggregated_dataset['log_income_mean'])-mu_mean)**2)
sigma_sq_highest = np.inner(np.array(aggregated_dataset['weight']), (np.array(aggregated_dataset['log_income_max'])-mu_highest)**2)

In [132]:
mle_result = df({'assumption': ["a) Lowest", "b) Mean", "c) Highest"], 
                'mu_hat': [mu_lowest, mu_mean, mu_highest],
                'sigma_sq_hat': [sigma_sq_lowest, sigma_sq_mean, sigma_sq_highest]})

In [175]:
MLE_mu = mu_highest
MLE_sigma = sigma_sq_highest ** (1/2)

In [133]:
mle_result

,assumption,mu_hat,sigma_sq_hat
0,a) Lowest,10.795785,2.239736
1,b) Mean,11.044071,0.871274
2,c) Highest,11.200075,0.967943


## Q.8

We can estimate the standard error of each moment using sample hessian.  
Hessian matrix equals the second derivative matrix of the log-likelihood function.  
I calculated the hessian matrix as follows:
$$\mathcal{H}(\hat{\mu}, \hat{\sigma}) = \begin{bmatrix}
\frac{1}{\hat{\sigma}^2} & \frac{1}{\hat{\sigma}^4}\frac{1}{n}\sum_i^n (\ln y_i - \hat{\mu}) \\
\frac{1}{\hat{\sigma}^4}\frac{1}{n}\sum_i^n (\ln y_i - \hat{\mu}) & -\frac{1}{2\hat{\sigma}^4} + \frac{1}{\hat{\sigma}^6}\frac{1}{n}\sum_i^n (\ln y_i - \hat{\mu})^2
\end{bmatrix}$$

In [176]:
H_11 = 1/sigma_sq_highest
H_12 = (1/(sigma_sq_highest)**2)*(np.inner(np.array(aggregated_dataset['weight']), (np.array(aggregated_dataset['log_income_max'])-mu_highest))) 
H_21 = (1/(sigma_sq_highest)**2)*(np.inner(np.array(aggregated_dataset['weight']), (np.array(aggregated_dataset['log_income_max'])-mu_highest))) 
H_22 = -(1/(2*(sigma_sq_highest)**2)) + (1/(sigma_sq_mean)**3)*np.inner(np.array(aggregated_dataset['weight']), (np.array(aggregated_dataset['log_income_max'])-mu_highest)**2)

Hessian = np.array([
    [H_11, H_12],
    [H_21, H_22]
    ])

Hessian_inv = np.linalg.inv(Hessian) 

print("estimated standard error of MoM mean esimator: ", (Hessian_inv[0][0] ** (1/2)))
print("estimated standard error of MoM variance esimator: ", Hessian_inv[1][1] ** (1/2))

estimated standard error of MoM mean esimator:  0.9838409787672148
estimated standard error of MoM variance esimator:  1.0370577249618638


## Q.9

It turns out that the point estimate depends on the assumptions for income bins. Especially, when we assumed the lowest income for each bin, estimate for the mean decreased by far compared to the other assumptions. Also, estimate for the variance increased. This is because under the log transformation, bin3's income level will vanish to zero. It makes the average level of income decrease and at the same time elevates the variance.  

According to the Cramer-Rao Efficiency, MLE estimator should be more efficient than the MoM estimator. However, my result turns shows the opposite.  
<span style = "color:red"> (I should go over this part.. I regarded the projection factor telling us the number of people in each bin. So the N is the total sum of the projection factor. Maybe this understanding could be incorrect. I will go over this part)</span>

## Q. 10

Using the data, we can construct a sample analogy for the vector of moment equations as follows:  

$$\bar{g}(\mu, \sigma) = \begin{bmatrix} 
Pr(\ln y_i\in bin_3|\mu, \sigma) - w_3 \\
Pr(\ln y_i\in bin_4|\mu, \sigma) - w_4 \\
Pr(\ln y_i\in bin_6|\mu, \sigma) - w_6 \\ 
\cdots \\
Pr(\ln y_i\in bin_{29}|\mu, \sigma) - w_{29} \\ 
Pr(\ln y_i\in bin_{30}|\mu, \sigma) - w_{30} \\ 
\end{bmatrix}$$  
where $\ln y_i \sim N(\mu, \sigma)$ and $w_b$ stands for the projection weight of $bin_b$. Hence we can write the Jacobian of the moment conditions as follows:  

$$\mathcal{J}(\mu, \sigma) = \mathbb{E}\begin{bmatrix} 
\frac{\partial Pr(\ln y_i\in bin_3|\mu, \sigma) - w_3}{\partial \mu} & \frac{\partial Pr(\ln y_i\in bin_3|\mu, \sigma) - w_3}{\partial \sigma}\\
\frac{\partial Pr(\ln y_i\in bin_4|\mu, \sigma) - w_4}{\partial \mu} & \frac{\partial Pr(\ln y_i\in bin_4|\mu, \sigma) - w_4}{\partial \sigma}\\
\frac{\partial Pr(\ln y_i\in bin_6|\mu, \sigma) - w_6}{\partial \mu} & \frac{\partial Pr(\ln y_i\in bin_6|\mu, \sigma) - w_6}{\partial \sigma}\\ 
\cdots & \cdots\\
\frac{\partial Pr(\ln y_i\in bin_{29}|\mu, \sigma) - w_{29}}{\partial \mu} & \frac{\partial Pr(\ln y_i\in bin_{29}|\mu, \sigma) - w_{29}}{\partial \sigma}\\ 
\frac{\partial Pr(\ln y_i\in bin_{30}|\mu, \sigma) - w_{30}}{\partial \mu} & \frac{\partial Pr(\ln y_i\in bin_{30}|\mu, \sigma) - w_{30}}{\partial \sigma}\\ 
\end{bmatrix}$$

## Q. 11

### First Stage Estimate

In [26]:
log_income_list = list(aggregated_dataset['log_income_amount'])
weight_list = list(aggregated_dataset['weight'])

def gmm_criterion(parameter):
    mu = parameter[0]
    sigma = parameter[1]
    benchmark_norm = scipy.stats.norm(mu, sigma)
    
    g_n_bar = []
    for i in range(len(log_income_list)):
        if i != len(log_income_list)-1:
            g_n_bar.append(benchmark_norm.cdf(log_income_list[i+1]) - benchmark_norm.cdf(log_income_list[i]) - weight_list[i])
        else:
            g_n_bar.append(benchmark_norm.cdf(100) - benchmark_norm.cdf(log_income_list[i]) - weight_list[i])
    
    g_n_bar = np.array(g_n_bar)
    criterion = g_n_bar @ g_n_bar
    
    return criterion

In [28]:
sample_init = np.array([10, 1.5])
res = minimize(gmm_criterion, sample_init, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.003787
         Iterations: 67
         Function evaluations: 132


In [46]:
print("One step GMM mean estimate: ", res.x[0])
print("One step GMM variance estimate: ", res.x[1]**2)

One step GMM mean estimate:  11.144447971538549
One step GMM variance estimate:  0.655830154115503


### Second Stage Estimate

#### calculate efficient weight matrix 

In [47]:
error_vector = []
mu = res.x[0]
sigma = res.x[1]
benchmark_norm = scipy.stats.norm(mu, sigma) 

error_vector = []
for i in range(len(log_income_list)):
    if i != len(log_income_list)-1:
        error_vector.append(benchmark_norm.cdf(log_income_list[i+1]) - benchmark_norm.cdf(log_income_list[i]) - weight_list[i])
    else:
        error_vector.append(benchmark_norm.cdf(100) - benchmark_norm.cdf(log_income_list[i]) - weight_list[i])

error_vector = np.array(error_vector)
ee = np.outer(error_vector, error_vector) / np.array(aggregated_dataset['projection_total']).sum()
W_hat = np.linalg.inv(ee) 

#### 2 step GMM

In [33]:
def gmm_criterion2(parameter):
    mu = parameter[0]
    sigma = parameter[1]
    benchmark_norm = scipy.stats.norm(mu, sigma)
    
    g_n_bar = []
    for i in range(len(log_income_list)):
        if i != len(log_income_list)-1:
            g_n_bar.append(benchmark_norm.cdf(log_income_list[i+1]) - benchmark_norm.cdf(log_income_list[i]) - weight_list[i])
        else:
            g_n_bar.append(benchmark_norm.cdf(100) - benchmark_norm.cdf(log_income_list[i]) - weight_list[i])
    
    g_n_bar = np.array(g_n_bar)
    criterion = g_n_bar @ W_hat @ g_n_bar
    
    return criterion

In [34]:
res2 = minimize(gmm_criterion, sample_init, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.003787
         Iterations: 67
         Function evaluations: 132


In [49]:
print("Two step GMM mean estimate: ", res2.x[0])
print("Two step GMM variance estimate: ", res2.x[1]**2)

Two step GMM mean estimate:  11.144447971538549
Two step GMM variance estimate:  0.655830154115503


### Calculate Standard Error

In [158]:
jacobian = []
for i in range(len(log_income_list)):

    def derivative_for_mu(mu):
        benchmark_norm = scipy.stats.norm(mu, 0.80983341)
        if i != len(log_income_list)-1:
            value = benchmark_norm.cdf(log_income_list[i+1]) - benchmark_norm.cdf(log_income_list[i])
        else:
            value = benchmark_norm.cdf(100) - benchmark_norm.cdf(log_income_list[i])
        return value

    def derivative_for_sigma(sigma):
        benchmark_norm = scipy.stats.norm(11.14444796, sigma)
        if i != len(log_income_list)-1:
            value = benchmark_norm.cdf(log_income_list[i+1]) - benchmark_norm.cdf(log_income_list[i])
        else:
            value = benchmark_norm.cdf(100) - benchmark_norm.cdf(log_income_list[i])
        return value
    
    der_mu = derivative(derivative_for_mu, 11.14444796, dx = 1e-6)
    der_sigma = derivative(derivative_for_sigma, 0.80983341, dx = 1e-6)
    
    jacobian.append([der_mu, der_sigma])

jacobian = np.array(jacobian)
cov_matrix = jacobian.transpose() @ W_hat @ jacobian
V_hat = -np.linalg.inv(cov_matrix) 

In [160]:
print("estimated standard error of GMM mean esimator: ", V_hat[0][0] ** (1/2))
print("estimated standard error of GMM variance esimator: ", V_hat[1][1] ** (1/2))

estimated standard error of GMM mean esimator:  1.1698346781362853e-12
estimated standard error of GMM variance esimator:  5.057059257142951e-13


## Q.12

In [178]:
method = ['MoM(Lowest Ass)', 'MLE (Highest Ass)', 'GMM']
mu =[MoM_mu, MLE_mu, res.x[0]]
sigma = [MoM_sigma, MLE_sigma, res.x[1]]
mean = [np.exp(MoM_mu), np.exp(MLE_mu), np.exp(res.x[0])]
p10 = [np.exp(scipy.stats.norm(MoM_mu, MoM_sigma).ppf(0.1)), np.exp(scipy.stats.norm(MLE_mu, MLE_sigma).ppf(0.1)), np.exp(scipy.stats.norm(res.x[0], res.x[1]).ppf(0.1))]
p90 = [np.exp(scipy.stats.norm(MoM_mu, MoM_sigma).ppf(0.9)), np.exp(scipy.stats.norm(MLE_mu, MLE_sigma).ppf(0.9)), np.exp(scipy.stats.norm(res.x[0], res.x[1]).ppf(0.9))]
std_err_mu = [MoM_std_err_mean, Hessian_inv[0][0] ** (1/2), V_hat[0][0] ** (1/2)]
std_err_sigma = [MoM_std_err_var, Hessian_inv[1][1] ** (1/2), V_hat[1][1] ** (1/2)]

result_table = df({'Method':method,
                  '𝜇̂':mu,
                  '𝜎̂':sigma,
                  'mean income': mean,
                  'p10 income': p10,
                  'p90 income': p90,
                  'std error of 𝜇̂': std_err_mu,
                  'std error of 𝜎̂': std_err_sigma})

result_table

,Method,𝜇̂,𝜎̂,mean income,p10 income,p90 income,std error of 𝜇̂,std error of 𝜎̂
0,MoM(Lowest Ass),10.795785,1.496575,48814.598532,7171.334326,332276.382840,8.033272e-04,1.700223e-03
1,MLE (Highest Ass),11.200075,0.983841,73135.951148,20727.827208,258052.486485,9.838410e-01,1.037058e+00
2,GMM,11.144448,0.809833,69178.677917,24504.319703,195299.830246,1.169835e-12,5.057059e-13


I reported the result of MoM under the lowest income assumption. And for the MLE result I followed the highest income assumption.  

Differences in the estimated parameters get amplified by transforming it into the dollor unit. From the different value in mean income level between MoM and MLE, we can check that the assumption for bin typed data makes a significant differences.  

As we can check from the column 'mean income', choosing lowest level in each bin can underestimate the income status whereas choosing the highest income can lead to an inflated result.  

When working with the bin typed data, I would rather take the <b>GMM approach</b>.
While constructing the moment equations, we did not have to set up any ridiculous assumption with respect to the income bin. We only deployed the distributional information, which could be more free from underestimating/overesimating issues caused by naive assumptions.  

<span style = "color:red"> Regarding the variance of each estimator, it really seems that I did not fully understand the data structure. I will go over this part again.</span>